# Physical implementation of E-comm

In [2]:
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

In [1]:
def connect():
    connection = psycopg2.connect(user="postgres",
                                  password="admin",
                                  host="localhost",
                                  port="5432",
                                  )
    connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT);
    return connection

## Creating Database e_comm

In [3]:
try:
    connection = connect()
    query = "CREATE DATABASE e_comm;"
    
    
    cursor = connection.cursor()
    
    # Executing a SQL query
    cursor.execute(query)
    print("Database sucessfully created")
except (Exception, Error) as error:
    print("Error while connecting to PostgreSQL", error)
finally:
    if (connection):
        cursor.close()
        connection.close()
        print('')
        print("PostgreSQL connection is closed")

Database sucessfully created

PostgreSQL connection is closed


#### Defining connect() method so we can we it laster through out our code

In [4]:
#Defining connect() method so we can we it laster through out our code to connect to the database
def connect():
    connection = psycopg2.connect(user="postgres",
                                  password="admin",
                                  host="localhost",
                                  port="5432",
                                  database="e_comm")
    
    return connection

### Creating tables in the database e_comm

In [14]:
import logging
try:
    connection = connect()
    
    cursor = connection.cursor()
    
    query_1 = '''
    CREATE TABLE category (
    id SERIAL PRIMARY KEY,
    name TEXT,
    description TEXT
    )
    '''
    cursor.execute(query_1)
    
    query_2 = '''
    CREATE TABLE brand (
    id SERIAL PRIMARY KEY,
    name TEXT,
    description TEXT
    )
    '''
    cursor.execute(query_2)
    
    query_3 = '''
    CREATE TABLE product (
    id SERIAL PRIMARY KEY,
    name TEXT,
    price MONEY,
    category_id INTEGER , 
    brand_id INTEGER,
    stock BOOLEAN,
    
    CONSTRAINT fk_product_category
    FOREIGN KEY(category_id) REFERENCES category(id),
    
    CONSTRAINT fk_product_brand
    FOREIGN KEY(brand_id) REFERENCES brand(id)
    
    )
    '''
    cursor.execute(query_3)
    
    query_4 = '''
    CREATE TABLE country (
    id SERIAL PRIMARY KEY,
    country_name TEXT
    )
    '''
    cursor.execute(query_4)
    
    query_5 = '''
    CREATE TABLE state (
    id SERIAL PRIMARY KEY,
    state_name TEXT,
    country_id INTEGER,
    
    CONSTRAINT fk_state_country
    FOREIGN KEY(country_id) REFERENCES country(id)
    )
    '''
    cursor.execute(query_5)
    
    query_6 = '''
    CREATE TABLE city (
    id SERIAL PRIMARY KEY,
    city_name TEXT,
    state_id INTEGER,
    
    CONSTRAINT fk_city_state
    FOREIGN KEY(state_id) REFERENCES state(id)
    )
    '''
    cursor.execute(query_6)
    
    query_7 = '''
    CREATE TABLE place (
    id SERIAL PRIMARY KEY,
    place_name TEXT,
    city_id INTEGER,
    
    CONSTRAINT fk_street_city
    FOREIGN KEY(city_id) REFERENCES city(id)
    )
    '''
    cursor.execute(query_7)
    
    
    query_8 = '''
    CREATE TABLE customer (
    id SERIAL PRIMARY KEY,
    address_id INTEGER,
    phone_number INTEGER,
    first_name TEXT,
    last_name TEXT,
    email TEXT,
    date_of_birth DATE,
    
    CONSTRAINT fk_customer_address_id
    FOREIGN KEY(address_id) REFERENCES place(id)
    
    )
    '''
    cursor.execute(query_8)
    
    
    query_9 = '''
    CREATE TABLE shipping_address (
    id SERIAL PRIMARY KEY,
    house_number TEXT,
    near_landmark TEXT,
    customer_id INTEGER,
    place_id INTEGER,
    
    CONSTRAINT fk_shiping_address_customer
    FOREIGN KEY(customer_id) REFERENCES customer(id),
    
    CONSTRAINT fk_shipping_address
    FOREIGN KEY(place_id) REFERENCES place(id)
    )
    '''
    cursor.execute(query_9)
    
    query_10 = '''
    CREATE TABLE payment_type (
    id SERIAL PRIMARY KEY,
    type TEXT
    )
    '''
    cursor.execute(query_10)
    
    query_11 = '''
    CREATE TABLE orders (
    id SERIAL PRIMARY KEY,
    customer_id INTEGER,
    payment_type_id INTEGER,
    order_date_time TIMESTAMP,
    shipping_address_id INTEGER,
    amount MONEY,
    
    CONSTRAINT fk_order_customer
    FOREIGN KEY(customer_id) REFERENCES customer(id),
    
    CONSTRAINT fk_order_payment_type
    FOREIGN KEY(payment_type_id) REFERENCES payment_type(id),
    
    CONSTRAINT fk_order_shipping_address
    FOREIGN KEY(shipping_address_id) REFERENCES shipping_address(id)
    
    )
    '''
    cursor.execute(query_11)
    
    query_12 = '''
    CREATE TABLE order_details (
    id SERIAL PRIMARY KEY,
    order_id INTEGER,
    product_id INTEGER,
    quantity SMALLINT,
    
    CONSTRAINT fk_order_details_order
    FOREIGN KEY(order_id) REFERENCES orders(id),
    
    CONSTRAINT fk_order_details_product
    FOREIGN KEY(product_id) REFERENCES product(id)
    )
    '''
    cursor.execute(query_12)
    
    query_13 = '''
    CREATE TABLE review (
    id SERIAL PRIMARY KEY,
    product_id INTEGER,
    customer_id INTEGER,
    review TEXT
    
    CONSTRAINT fk_review_customer
    FOREIGN KEY(customer_id) REFERENCES customer(id),
    
    CONSTRAINT fk_review_product
    FOREIGN KEY(product_id) REFERENCES product(id)
    )
    '''
    cursor.execute(query_13)
    
    
    connection.commit()
    
    
    
    print("Tables sucessfully created")
except (Exception,psycopg2.Error) as error:
    #print("Error while connecting to PostgresSQL: ",error)
    #print(error)
    logging.debug("An exception was thrown!", exc_info=True)
    logging.info("An exception was thrown!", exc_info=True)
    logging.warning("An exception was thrown!", exc_info=True)
finally:
    if(connection):
        cursor.close()
        connection.close()
        print('')
        print("PostgreSQL connection is closed")

Tables sucessfully created

PostgreSQL connection is closed


### Populating the database with data

In [20]:
import pandas as pd

In [21]:
tables = ['category','brand','product','country','state','city','place','customer','shipping_address','payment_type','orders','order_details','review']

In [29]:

connection=connect()
cursor = connection.cursor()

try:
    for table in tables:
        df = pd.read_excel ('data for E-comm.xlsx',sheet_name=table)
        for columns in df.columns:
            #print(columns)
            try:
                if df[columns].dtypes =='float':
                    print(str(columns))
                    df[columns]=df[columns].astype('int32')
            except:
                print(str(columns))
                pass

        data = [list(x) for x in df.itertuples(index=False)]
        query = "INSERT INTO " + str(table)+" VALUES(" +'%s'+',%s'*(len(df.columns)-1) + ');'

        print(query)
        for item in data:
            to_push=item
            for i,ele in enumerate(item):
                if str(ele)=='nan':
                    to_push[i]=None
                    
            cursor.execute(query,to_push)
            connection.commit()
        print("Data sucessufully inserted of table "+str(table))
        
except(Exception,psycopg2.Error) as e:
    print(e)
    logging.debug("An exception was thrown!", exc_info=True)
    logging.info("An exception was thrown!", exc_info=True)
    logging.warning("An exception was thrown!", exc_info=True)

finally:
    if(connection):
        cursor.close()
        print('')
        print("PostgreSQL connection is closed")

id
INSERT INTO category VALUES(%s,%s,%s);
Data sucessufully inserted of table category
id
INSERT INTO brand VALUES(%s,%s,%s);
Data sucessufully inserted of table brand
id
price
category_id
brand_id
INSERT INTO product VALUES(%s,%s,%s,%s,%s,%s);
Data sucessufully inserted of table product
id
INSERT INTO country VALUES(%s,%s);
Data sucessufully inserted of table country
id
state_name
country
INSERT INTO state VALUES(%s,%s,%s);
Data sucessufully inserted of table state
id
state_name
INSERT INTO city VALUES(%s,%s,%s);
Data sucessufully inserted of table city
id
city_id
INSERT INTO place VALUES(%s,%s,%s);
Data sucessufully inserted of table place
id
address_id
phone_number
INSERT INTO customer VALUES(%s,%s,%s,%s,%s,%s,%s);
Data sucessufully inserted of table customer
id
customer_id
place_id
INSERT INTO shipping_address VALUES(%s,%s,%s,%s,%s);
Data sucessufully inserted of table shipping_address
id
INSERT INTO payment_type VALUES(%s,%s);
Data sucessufully inserted of table payment_type
id
cu